In [1]:
using IJuliaPortrayals

In [2]:
using MXNet

In [3]:
# CNN構築

# input
data = mx.Variable(:data)

# first conv
conv1 = @mx.chain mx.Convolution(data=data, kernel=(5,5), num_filter=32)  =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# second conv
conv2 = @mx.chain mx.Convolution(data=conv1, kernel=(5,5), num_filter=64) =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# first fully-connected
fc1   = @mx.chain mx.Flatten(data=conv2) =>
                  mx.FullyConnected(num_hidden=1024) =>
                  mx.Activation(act_type=:relu)

dp_fc1 = mx.Dropout(fc1, p=0.5)

# second fully-connected
fc2   = mx.FullyConnected(data=dp_fc1, num_hidden=10)

# softmax loss
cnn = mx.SoftmaxOutput(data=fc2, name=:softmax)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x00007f828448a390))

In [4]:
GraphViz(mx.to_graphviz(cnn))

IJuliaPortrayals.GraphViz("digraph \"Network Visualization\" {\nnode [fontsize=10];\nedge [fontsize=10];\n\"convolution0\"  [label=\"convolution0\\nConvolution\\nkernel=(5,5)\\nstride=(1,1)\\nn-filter=32\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fb8072\",shape=box,penwidth=2,height=0.8034,color=\"#941305\"];\n\"activation0\"  [label=\"activation0\\nActivation\\nact-type=relu\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#ffffb3\",shape=box,penwidth=2,height=0.8034,color=\"#999900\"];\n\"pooling0\"  [label=\"pooling0\\nPooling\\ntype=max\\nkernel=(2,2)\\nstride=(2,2)\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#80b1d3\",shape=box,penwidth=2,height=0.8034,color=\"#275372\"];\n\"convolution1\"  [label=\"convolution1\\nConvolution\\nkernel=(5,5)\\nstride=(1,1)\\nn-filter=64\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fb8072\",shape=box,penwidth=2,height=0.8034,color=\"#941305\"];\n\"activation1\"  [label=\"activation1\\nActivation\\nact-type=relu\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#ffffb3\",shape=box,penwidth=2,height=0.8034,color=\"#999900\"];\n\"pooling1\"  [label=\"pooling1\\nPooling\\ntype=max\\nkernel=(2,2)\\nstride=(2,2)\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#80b1d3\",shape=box,penwidth=2,height=0.8034,color=\"#275372\"];\n\"flatten0\"  [label=\"flatten0\\nFlatten\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fdb462\",shape=box,penwidth=2,height=0.8034,color=\"#975102\"];\n\"fullyconnected0\"  [label=\"fullyconnected0\\nFullyConnected\\nnum-hidden=1024\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fb8072\",shape=box,penwidth=2,height=0.8034,color=\"#941305\"];\n\"activation2\"  [label=\"activation2\\nActivation\\nact-type=relu\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#ffffb3\",shape=box,penwidth=2,height=0.8034,color=\"#999900\"];\n\"dropout0\"  [label=\"dropout0\\nDropout\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fccde5\",shape=box,penwidth=2,height=0.8034,color=\"#90094e\"];\n\"fullyconnected1\"  [label=\"fullyconnected1\\nFullyConnected\\nnum-hidden=10\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#fb8072\",shape=box,penwidth=2,height=0.8034,color=\"#941305\"];\n\"softmax\"  [label=\"softmax\\nSoftmaxOutput\",style=\"rounded,filled\",fixedsize=true,width=1.3,fillcolor=\"#b3de69\",shape=box,penwidth=2,height=0.8034,color=\"#597d1c\"];\n\"activation0\" -> \"convolution0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"pooling0\" -> \"activation0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"convolution1\" -> \"pooling0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"activation1\" -> \"convolution1\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"pooling1\" -> \"activation1\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"flatten0\" -> \"pooling1\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"fullyconnected0\" -> \"flatten0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"activation2\" -> \"fullyconnected0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"dropout0\" -> \"activation2\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"fullyconnected1\" -> \"dropout0\"  [arrowtail=open,color=\"#737373\",dir=back];\n\"softmax\" -> \"fullyconnected1\"  [arrowtail=open,color=\"#737373\",dir=back];\n}\n","dot","svg")

In [5]:
# データ取得（データプロバイダ生成）
batch_size = 100
# include(Pkg.dir("MXNet", "examples", "mnist", "mnist-data.jl"))
# train_provider, eval_provider = get_mnist_providers(batch_size)
data_name = :data
label_name = :softmax_label
flat=false
train_provider = mx.MNISTProvider(image="MNIST_data/train-images-idx3-ubyte",
                                  label="MNIST_data/train-labels-idx1-ubyte",
                                  data_name=data_name, label_name=label_name,
                                  batch_size=batch_size, shuffle=true, flat=flat, silent=true)
eval_provider = mx.MNISTProvider(image="MNIST_data/t10k-images-idx3-ubyte",
                                 label="MNIST_data/t10k-labels-idx1-ubyte",
                                 data_name=data_name, label_name=label_name,
                                 batch_size=batch_size, shuffle=false, flat=flat, silent=true)


MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Void} @0x00007f82848a5910),Tuple{Symbol,Tuple}[(:data,(28,28,1,100))],Tuple{Symbol,Tuple}[(:softmax_label,(100,))],100,true,true)

In [6]:
# モデル構築・最適化

# モデル setup
model = mx.FeedForward(cnn, context=mx.cpu())

# optimization algorithm
optimizer = mx.SGD(lr=0.05, momentum=0.9, weight_decay=0.00001)

# save-checkpoint callback
save_checkpoint = mx.do_checkpoint("MNIST_CNN3")

# fit parameters
mx.fit(model, optimizer, train_provider, n_epoch=30, eval_data=eval_provider, callbacks=[save_checkpoint])

INFO: Start training on [CPU0]
INFO: Initializing parameters...
INFO: Creating KVStore...
INFO: Start training...
INFO: == Epoch 001 ==========
INFO: ## Training summary
INFO:           accuracy = 0.7142
INFO:               time = 223.0584 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9740
INFO: Saved checkpoint to 'MNIST_CNN3-0001.params'
INFO: == Epoch 002 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9804
INFO:               time = 208.5880 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9865
INFO: Saved checkpoint to 'MNIST_CNN3-0002.params'
INFO: == Epoch 003 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9869
INFO:               time = 211.7859 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9892
INFO: Saved checkpoint to 'MNIST_CNN3-0003.params'
INFO: == Epoch 004 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9898
INFO:               time = 206.5210 seconds
INFO: ## Va

In [7]:
# 予測
probs = mx.predict(model, eval_provider)

10x10000 Array{Float32,2}:
 6.6912e-25   1.84421e-19  3.34017e-14  …  6.27268e-23  5.81799e-18
 7.64353e-21  6.56743e-20  1.0             8.11345e-25  2.53667e-19
 5.96325e-20  1.0          7.18634e-14     1.18043e-28  3.24815e-19
 7.11701e-20  5.54137e-23  1.69543e-18     2.38322e-18  1.05279e-21
 1.03662e-19  1.21309e-25  8.94513e-13     1.3275e-28   1.67418e-19
 2.30987e-25  1.73122e-29  1.34319e-14  …  1.0          5.61854e-19
 1.45137e-27  7.20162e-20  9.96349e-14     2.55676e-18  1.0        
 1.0          1.75189e-23  1.66512e-12     7.50446e-26  2.32328e-25
 6.29969e-28  4.04601e-22  9.14715e-14     8.67362e-15  5.61445e-20
 2.57974e-17  1.27717e-27  3.43039e-16     4.33827e-21  3.61943e-22

In [8]:
# 予測精度確認

# collect all labels from eval data
labels = Array[]
for batch in eval_provider
    push!(labels, copy(mx.get(eval_provider, batch, :softmax_label)))
end
labels = cat(1, labels...)

# Now we use compute the accuracy
correct = 0
for i = 1:length(labels)
    # labels are 0...9
    if indmax(probs[:,i]) == labels[i]+1
        correct += 1
    end
end
accuracy = 100correct/length(labels)
println(mx.format("Accuracy on eval set: {1:.2f}%", accuracy))

Accuracy on eval set: 99.43%


In [9]:
batch = first(eval_provider)
# images0 = copy(mx.get(eval_provider, batch, :data))
# size(images0)
image = copy(mx.get(eval_provider, batch, :data))[:,:,:,1:1]

# all(x->0.0<=x<=1.0,vec(image))
# => true

# provider = mx.ArrayDataProvider(images[:,1:1])
provider = mx.ArrayDataProvider(image)

MXNet.mx.ArrayDataProvider(Array{Float32,N}[
784x1 Array{Float32,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0],[:data],Array{Float32,N}[],Symbol[],1,1,false,0.0f0,0.0f0,[mx.NDArray(28,28,1,1)],MXNet.mx.NDArray[])

In [10]:
mx.predict(model, provider)

10x1 Array{Float32,2}:
 6.6912e-25 
 7.64353e-21
 5.96325e-20
 7.11701e-20
 1.03662e-19
 2.30987e-25
 1.45137e-27
 1.0        
 6.29969e-28
 2.57974e-17

In [11]:
import JSON

function classify(a::Vector{Float32})
    image = reshape(a, (28, 28, 1, 1))
    # classify
    result = mx.predict(model, mx.ArrayDataProvider(image))
    return JSON.json(vec(result))
end

function classify(a::Vector)
    classify(convert(Vector{Float32}, a))
end

function classify(s::AbstractString)
    classify(JSON.parse(s))
end

classify (generic function with 3 methods)

In [12]:
HTML(open(readall, "classify_canvas.html"))

HTML{ASCIIString}("<style type=\"text/css\">\ncanvas { border: 1px solid black; }\n</style>\n<div id=\"board\">\n<canvas id=\"myCanvas\" width=\"112px\" height=\"112px\">Sorry, your browser doesn't support canvas technology.</canvas>\n<p>\n<button id=\"classify\" onclick=\"classify()\">Classify</button>\n<button id=\"clear\" onclick=\"myClear()\">Clear</button>\n</p><p>\nResult: <input type=\"text\" id=\"result_output\" size=\"5\" value=\"\"><br><textarea id=\"result_detail\" cols=\"20\" rows=\"10\"></textarea>\n</p>\n</div>\n<script type=\"text/javascript\">\nfunction getOffset(el) {\n\treturn el.getBoundingClientRect();\n}\nfunction init() {\n\tvar myCanvas = document.getElementById(\"myCanvas\");\n\t// var curColor = void 0;\n\tif (myCanvas) {\n\t\tvar isDown = false;\n\t\tvar ctx = myCanvas.getContext(\"2d\");\n\t\tvar canvasX, canvasY;\n\t\tctx.lineWidth = 8;\n\t\tfunction onMouseDown(e) {\n\t\t\tisDown = true;\n\t\t\tctx.beginPath();\n\t\t\tvar parentOffset = getOffset(myCanvas.parentNode); \n\t\t\tcanvasX = e.pageX - parentOffset.left;\n\t\t\tcanvasY = e.pageY - parentOffset.top;\n\t\t\tctx.moveTo(canvasX, canvasY);\n\t\t}\n\t\tmyCanvas.addEventListener(\"mousedown\", onMouseDown, false);\n\t\tfunction onMouseMove(e) {\n\t\t\tif(isDown != false) {\n\t\t\t\tvar parentOffset = getOffset(myCanvas.parentNode); \n\t\t\t\tcanvasX = e.pageX - parentOffset.left;\n\t\t\t\tcanvasY = e.pageY - parentOffset.top;\n\t\t\t\tctx.lineTo(canvasX, canvasY);\n\t\t\t\t// ctx.strokeStyle = curColor;\n\t\t\t\tctx.stroke();\n\t\t\t}\n\t\t}\n\t\tmyCanvas.addEventListener(\"mousemove\", onMouseMove, false);\n\t\tfunction onMouseUp(e) {\n\t\t\tisDown = false;\n\t\t\tctx.closePath();\n\t\t}\n\t\tmyCanvas.addEventListener(\"mouseup\", onMouseUp, false);\n\t}\n}\ninit();\nfunction indMax(a) {\n\tvar res = -1;\n\tvar val = Number.NEGATIVE_INFINITY;\n\tfor (var i = 0, l = a.length; i < l; i++) {\n\t\tif (a[i] > val) {\n\t\t\tval = a[i];\n\t\t\tres = i;\n\t\t}\n\t}\n\treturn res;\n}\nfunction handle_output(out) {\n\tconsole.log(out);\n\tvar arr = JSON.parse(JSON.parse(out.content.data[\"text/plain\"]));\n\tvar result = indMax(arr);\n\tvar detail = arr.map(function (v, i) {return i + ': ' + v;});\n\tdocument.getElementById(\"result_output\").value = result;\n\tdocument.getElementById(\"result_detail\").value = detail.join(\"\\n\");\n}\nfunction classify() {\n\tvar kernel = IPython.notebook.kernel;\n\tvar myCanvas = document.getElementById(\"myCanvas\");\n\tvar ctx = myCanvas.getContext(\"2d\");\n\tvar data = [];\n\tfor (var y = 0; y < 28; ++y){\n\t\tfor (var x = 0; x < 28; ++x){\n\t\t\tvar imageData = ctx.getImageData(x * 4, y * 4, 4, 4);\n\t\t\tvar pixel = 0;\n\t\t\tfor (var i = 3; i < 64; i += 4) pixel += imageData.data[i];\n\t\t\tdata.push(pixel / 4080);\t// 4080 = 255*16\n\t\t}\n\t}\n\tdocument.getElementById(\"result_output\").value = \"\";\n\tconsole.log(data);\n\tkernel.execute('classify(' + JSON.stringify(data) + ')',  { 'iopub': {'output': handle_output}}, {silent: false});\n}\nfunction myClear() {\n\tvar myCanvas = document.getElementById(\"myCanvas\");\n\tmyCanvas.getContext(\"2d\").clearRect(0, 0, myCanvas.width, myCanvas.height);\n}\n</script>")